<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/Exemplo_de_Self_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemplo de self attention

In [ ]:
import torch 

In [ ]:
# 00 -- ENTRADA:
text = 'Quero um cartão'
text_tokens = ['Quero', 'um', 'cartão']

# 01 -- IDs:
text_ids = [27,11,83]
VOCAB = 100 # Quantidade de palavras

# 02 -- DIMENSÃO DOS EMBEDDINGS = 4:
D = 4

# 03 -- COMPRIMENTO DA SEQUÊNCIA:
L = 3

# 04 -- MATRIZ DE EMBEDDINGS LxD:
text_emb = torch.tensor([
                         [0,1,2,0], 
                         [1,0,1,1],
                         [1,0,1,2],
                        ]).long()

# 05 -- PRINT - SANITY CHECK:
for i in zip(text_tokens, text_ids, text_emb):
    print(f'Palavra: {i[0] :<6} -- ID: {i[1]} -- Embedding: {i[2]}')

Palavra: Quero  -- ID: 27 -- Embedding: tensor([0, 1, 2, 0])
Palavra: um     -- ID: 11 -- Embedding: tensor([1, 0, 1, 1])
Palavra: cartão -- ID: 83 -- Embedding: tensor([1, 0, 1, 2])


In [ ]:
# # 06 -- USAR O nn.EMBEDDING PyTorch para CONSTRUIR AS MATRIZ DE PESOS Q, K, V:

# embs  = torch.nn.Embedding(VOCAB, D)

# # tensor com zeros_like
# embeddings = torch.zeros(len(text_ids) * D).reshape(len(text_ids), D) 

# print(f'Shape da matriz de embeddings: {embeddings.shape}\n')

# print('Texto\t\t\t     Matriz de embeddings\n')
# for i , token in enumerate(text_ids):
#     embeddings[i] = embs(torch.tensor(token))
# embeddings

In [ ]:
# 06 -- MATRIZ DE PESOS, Q, K, V --> DxD (INICIADOS ARBITRARIAMENTE):

Qw = torch.tensor([
                   [1,  0, -1,  1], 
                   [0,  1,  0, -2],
                   [3,  1, -2,  0],
                   [0, -2,  1,  3],
                ])

Kw = torch.tensor([
                   [3, -1,  0, -1], 
                   [0,  1, -1,  0],
                   [3,  1,  0, -1],
                   [-1, 0,  1,  3],
                 ])

Vw = torch.tensor([
                   [-2, 0,  3,  0], 
                   [-1, 1, -2, -1],
                   [-2, 1,  2,  3],
                   [3,  2,  1, -2],
                 ])

In [ ]:
# 07 -- OBTER Q, K e V:
print(f'Shape da entrada: {text_emb.size()} -- Shape das Matrizes de pesos Qw, Kw e Vw: {Qw.size()}')

Shape da entrada: torch.Size([3, 4]) -- Shape das Matrizes de pesos Qw, Kw e Vw: torch.Size([4, 4])


In [ ]:
K =  torch.einsum('ab, bc -> ac', text_emb, Kw) # text_emb @ Kw
V =  torch.einsum('ab, bc -> ac', text_emb, Vw) # text_emb @ Vw
Q =  torch.einsum('ab, bc -> ac', text_emb, Qw) # text_emb @ Qw

In [ ]:
K

tensor([[ 6,  3, -1, -2],
        [ 5,  0,  1,  1],
        [ 4,  0,  2,  4]])

In [ ]:
V

tensor([[-5,  3,  2,  5],
        [-1,  3,  6,  1],
        [ 2,  5,  7, -1]])

In [ ]:
Q

tensor([[ 6,  3, -4, -2],
        [ 4, -1, -2,  4],
        [ 4, -3, -1,  7]])

In [ ]:
# 08 -- CALCULAR O SCORE:
QKT = torch.einsum('ab, cb -> ac', Q, K) # Q @ K.T
QKT

tensor([[53, 24,  8],
        [15, 22, 28],
        [ 2, 26, 42]])

In [ ]:
# 09 -- SOFTMAX:
S = torch.nn.Softmax(dim=-1)
S(QKT.float())

tensor([[1.0000e+00, 2.5437e-13, 2.8625e-20],
        [2.2547e-06, 2.4726e-03, 9.9753e-01],
        [4.2484e-18, 1.1254e-07, 1.0000e+00]])

In [ ]:
# 10 -- SIMPLIFICAR S - POR SIMPLICIDADE DE ENTENDIMENTO:
Softmax  = torch.tensor([
                         [1,    0,    0  ], 
                         [0.01, 0.09, 0.9],
                         [0,    0,    1  ],
                 ])

In [ ]:
# 11 -- SOFTMAX * V (EXTRATOR DE FEAT):
SV = torch.einsum('ab, bc -> ac', Softmax,V.float())
SV 

tensor([[-5.0000,  3.0000,  2.0000,  5.0000],
        [ 1.6600,  4.8000,  6.8600, -0.7600],
        [ 2.0000,  5.0000,  7.0000, -1.0000]])

In [ ]:
# 12 -- MULTIPLICAR PELA MATRIZ DE PROJEÇÃO Wo

# PESOS DA Wo (INICIADOS ARBITRARIAMENTE)
Wo = torch.tensor([
                   [2,  0, -1,  1], 
                   [1, -1, -2,  1],
                   [2,  0,  1,  0],
                   [0, -2,  1, -1],
                 ])
Wo.shape

torch.Size([4, 4])

In [ ]:
## 13 -- SV @ Wo = MATRIZ DE ATENÇÃO:

ATT = torch.einsum('ab, bc -> ac', SV, Wo.float()) # SV @ Wo
ATT

tensor([[ -3.0000, -13.0000,   6.0000,  -7.0000],
        [ 21.8400,  -3.2800,  -5.1600,   7.2200],
        [ 23.0000,  -3.0000,  -6.0000,   8.0000]])

# Fim do Notebook